In [3]:
import csv

# Path to your .txt file
input_file_path = '/users/minseoksong/Desktop/obsidian_data.txt'
# Path to the output .sql script
output_file_path = '/users/minseoksong/Desktop/obsidian_data.sql'

# Create table structure
table_creation = """CREATE TABLE artifacts (
    ID TEXT PRIMARY KEY,
    mass FLOAT,
    type TEXT,
    site TEXT,
    element_Rb INT,
    element_Sr INT,
    element_Y INT,
    element_Zr INT
);
"""

with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
    reader = csv.reader(input_file)
    
    # Write the table creation SQL to the output file
    output_file.write(table_creation)
    output_file.write('\n')
    
    headers = next(reader)  # skip the header row
    for row in reader:
        id_, mass, type_, site, rb, sr, y, zr = row
        sql = f"INSERT INTO artifacts (ID, mass, type, site, element_Rb, element_Sr, element_Y, element_Zr) VALUES ('{id_}', {mass}, '{type_}', '{site}', {rb}, {sr}, {y}, {zr});\n"
        output_file.write(sql)

print(f"SQL script has been written to {output_file_path}")


SQL script has been written to /users/minseoksong/Desktop/obsidian_data.sql


In [4]:
import pandas as pd
import mysql.connector
import warnings

def fetch_data(query):
    # Suppress the UserWarning
    warnings.simplefilter(action='ignore', category=UserWarning)
    
    # Establish the connection
    cnx = mysql.connector.connect(user='root', host='127.0.0.1', database='mimnseoksong')
    
    # Fetch the data into a DataFrame
    df = pd.read_sql(query, cnx)
    
    # Close the connection
    cnx.close()
    
    # Return the DataFrame
    return df

How does the data look like?

In [6]:
fetch_data("""
SELECT *
FROM artifacts      
""")

,ID,mass,type,site,element_Rb,element_Sr,element_Y,element_Zr
0,288275.002a,0.502,Blade,Ali Kosh,238,45,29,334
1,288275.002aa,0.227,Flake,Ali Kosh,234,44,28,325
2,288275.002ab,0.188,Flake,Ali Kosh,255,50,32,337
3,288275.002ac,0.153,Flake,Ali Kosh,231,46,28,327
4,288275.002ad,0.102,Blade,Ali Kosh,252,49,31,331
...,...,...,...,...,...,...,...,...
645,297099a,1.838,Retouched Blade,Chagha Sefid,230,44,29,330
646,297103a,0.314,Blade,Chagha Sefid,237,47,30,329
647,297108d,0.866,Flakes,Chagha Sefid,238,45,29,337
648,297109a,1.624,Core/Fragment,Chagha Sefid,225,42,27,327


Disclaimer: given the small number of dataset, using with R might be suitable. Some queries needs to precede with data cleaning.

How many types do we have?

In [8]:
fetch_data("""
SELECT distinct type, count(type)
FROM artifacts
group by type
""")

,type,count(type)
0,Blade,376
1,Flake,202
2,Core fragment?,1
3,Core fragment? Flake?,1
4,Blade/Flake,3
5,Core,15
6,NA,1
7,Flake/Core,1
8,Core fragment,3
9,Cores and frags,2


Lengths of ID?

In [11]:
fetch_data("""
SELECT distinct length(ID), count(length(ID))
FROM artifacts
group by length(ID)
""")

,length(ID),count(length(ID))
0,11,21
1,12,51
2,8,181
3,7,397


What is the average mass of obsidians for each type and site? Rank based on the average mass.

In [21]:
fetch_data("""
SELECT type, site, AVG(mass)
FROM artifacts
group by type, site
LIMIT 15
""")

,type,site,AVG(mass)
0,Blade,Ali Kosh,0.301556
1,Flake,Ali Kosh,0.490274
2,Core fragment?,Ali Kosh,0.713000
3,Core fragment? Flake?,Ali Kosh,0.264000
4,Blade/Flake,Ali Kosh,0.555667
5,Core,Ali Kosh,5.623666
6,NA,Ali Kosh,0.272000
7,Flake/Core,Ali Kosh,2.256000
8,Blade,Ali Kosh/Chaga Sefid,0.101000
9,Blade,Hulailan Tepe Guran,0.284000


In [26]:
fetch_data("""
SELECT type, site, AVG(mass) as average_mass, RANK() OVER(ORDER BY AVG(mass) DESC) as rank_by_mass
FROM artifacts
group by type, site
LIMIT 10
""")

,type,site,average_mass,rank_by_mass
0,Core,Ali Kosh,5.623666,1
1,Cores and fragments,Chagha Sefid,4.795000,2
2,core,Chagha Sefid,2.666000,3
3,Flake/Core,Ali Kosh,2.256000,4
4,Distal end of prismatic blade?,Chagha Sefid,2.040000,5
5,Flake,Chagha Sefid,1.953471,6
6,Retouched Blade,Chagha Sefid,1.838000,7
7,Core/Fragment,Chagha Sefid,1.624000,8
8,Retouched blades,Chagha Sefid,1.363000,9
9,Used flake,Chagha Sefid,1.257000,10


Rank the average mass for the top two sites in frequency.

In [31]:
fetch_data("""
WITH RelevantSites AS (
    SELECT site
    FROM artifacts
    GROUP BY site
    ORDER BY COUNT(*) DESC
    LIMIT 2
)

SELECT a.type, a.site, AVG(a.mass) as average_mass,
       RANK() OVER(PARTITION BY a.site ORDER BY AVG(a.mass) DESC) as rank_by_mass
FROM artifacts a
JOIN RelevantSites rs ON a.site = rs.site
GROUP BY a.type, a.site
""")

,type,site,average_mass,rank_by_mass
0,Core,Ali Kosh,5.623666,1
1,Flake/Core,Ali Kosh,2.256000,2
2,Core fragment?,Ali Kosh,0.713000,3
3,Blade/Flake,Ali Kosh,0.555667,4
4,Flake,Ali Kosh,0.490274,5
5,Blade,Ali Kosh,0.301556,6
6,NA,Ali Kosh,0.272000,7
7,Core fragment? Flake?,Ali Kosh,0.264000,8
8,Cores and fragments,Chagha Sefid,4.795000,1
9,core,Chagha Sefid,2.666000,2


Which sites have the highest average concentrations of each element (Rb, Sr, Y, Zr)? (two ways)

In [53]:
fetch_data("""
SELECT site, RANK() OVER(ORDER BY AVG(element_Rb+element_Sr+element_Y+element_Zr) desc)
FROM artifacts
group by site
""")

,site,RANK() OVER(ORDER BY AVG(element_Rb+element_Sr+element_Y+element_Zr) desc)
0,Ali Kosh/Chaga Sefid,1
1,Hulailan Tepe Guran,2
2,Ali Kosh,3
3,Chagha Sefid,4


In [52]:
fetch_data("""
WITH ElementSums AS (
    SELECT site, 
           AVG(element_Rb + element_Sr + element_Y + element_Zr) as avg_element_sum
    FROM artifacts
    GROUP BY site
)

SELECT site, 
       RANK() OVER(ORDER BY avg_element_sum DESC) as rank_by_avg_element_sum
FROM ElementSums
""")

,site,rank_by_avg_element_sum
0,Ali Kosh/Chaga Sefid,1
1,Hulailan Tepe Guran,2
2,Ali Kosh,3
3,Chagha Sefid,4


Average mass of the top two site with highest average concentrations of each element (Rb, Sr, Y, Zr)

In [72]:
fetch_data("""
WITH toptwo_sites as(
SELECT site, RANK() OVER(ORDER BY AVG(element_Rb+element_Sr+element_Y+element_Zr) desc)
FROM artifacts
group by site
LIMIT 2
)

SELECT a.site, AVG(a.element_Rb+a.element_Sr+a.element_Y+a.element_Zr) as avg_amt
FROM artifacts a
JOIN toptwo_sites t ON a.site = t.site
group by a.site
""")

,site,avg_amt
0,Ali Kosh/Chaga Sefid,735.0
1,Hulailan Tepe Guran,680.0
